In [1]:
# We install the package through devtools::install_github
# This requires the "devtools" package to be installed
install.packages("devtools",  INSTALL_opts = c('--no-lock'))
library(devtools)
devtools::install_github("TobiasRoikjer/PtDAlgorithms")

Installing package into ‘/home/tobias/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)

Loading required package: usethis





✔  checking for file ‘/tmp/RtmpThPsM1/remotes53755078a75/TobiasRoikjer-PtDAlgorithms-a4fbecd/DESCRIPTION’ (408ms)
─  preparing ‘ptdalgorithms’:
✔  checking DESCRIPTION meta-information
─  cleaning src
─  checking for LF line-endings in source and make files and shell scripts (426ms)
─  checking for empty or unneeded directories
   Omitted ‘LazyData’ from DESCRIPTION
─  building ‘ptdalgorithms_1.0.0.tar.gz’
   


Installing package into ‘/home/tobias/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



In [2]:
# Import the functions in the library
library(ptdalgorithms)

# The library supports setting seeds for some of its
# sampling capability, as any other R package
set.seed(1234)

In [3]:
# While the R api can create graphs, this is *slow* as the C++ binding layer
# through Rcpp is slow when invoking many functions, and since R is a slow,
# interpreted language
# Clone or download the code, and include these files in the repository!
# Make SURE that the version of the downloaded code is the same as the
# installed R library!! Otherwise it may crash randomly
# The file has comments and is easy to understand, so you should be able
# to defined you own cool construction functions

Rcpp::sourceCpp("./isolation_migration_model.cpp")

In [15]:
# Construction of IM model and computation of expectations

n1 <- 8
n2 <- 8

time <- proc.time()[3]
im_graph <- construct_im_graph(n1,n2,0.1,0.1)
ancestral_graph <- construct_ancestral_graph(n1, n2)

cat(paste("Construction took ", proc.time()[3] - time, "seconds\n"))

time <- proc.time()[3]

ancestral_start_probability <-
    start_prob_from_im(ancestral_graph, im_graph, stop_probability(im_graph, time = 1.5))

cat(paste("Finding stopping time took ", proc.time()[3] - time, "seconds\n"))


full_im_expectation <- matrix(nrow=n1+1,ncol=n2+1)
truncation_im_expectation <- matrix(nrow=n1+1,ncol=n2+1)
ancestral_expectation <- matrix(nrow=n1+1,ncol=n2+1)

time <- proc.time()[3]
# This is just to get the timing, not necessary
k <- expectation(im_graph)

cat(paste("Moment graph took ", proc.time()[3] - time, "seconds\n"))

time <- proc.time()[3]
for (i in 0:n1) {
  for (j in 0:n2) {
        full_im_expectation[i+1, j+1]<- expectation(im_graph, rewards_at(im_graph, i,j,n1,n2))
        truncation_im_expectation[i+1, j+1]<-
          sum(distribution_context_stop_probability(ctx) *
              expected_waiting_time(im_graph, rewards_at(im_graph, i,j,n1,n2)))
        ancestral_expectation[i+1, j+1]<-
          sum(ancestral_start_probability *
              expected_waiting_time(ancestral_graph, rewards_at(ancestral_graph, i,j,n1,n2)))
    }
}

cat(paste("Expectations took ", proc.time()[3] - time, "seconds\n"))
full_im_expectation - truncation_im_expectation + ancestral_expectation


Construction took  8.23700000000002 seconds
Finding stopping time took  182.073 seconds
Moment graph took  475.957 seconds
Expectations took  118.025 seconds


0.0000000,1.94437055,0.91728479,0.57500500,0.405301445,0.305973594,0.244728461,0.216488684,0.43214155
1.9443705,0.05601000,0.03866822,0.03280713,0.029739771,0.027952386,0.027373938,0.029861369,0.08763481
0.9172848,0.03866822,0.02326801,0.01852383,0.016371045,0.015327841,0.015176856,0.017021119,0.05885976
0.5750050,0.03280713,0.01852383,0.01414322,0.012209738,0.011305574,0.011164441,0.012576153,0.04919712
0.4053014,0.02973977,0.01637104,0.01220974,0.010368679,0.009498186,0.009309170,0.010399057,0.04499304
0.3059736,0.02795239,0.01532784,0.01130557,0.009498186,0.008616454,0.008355782,0.009165145,0.04312298
0.2447285,0.02737394,0.01517686,0.01116444,0.009309170,0.008355782,0.007979437,0.008494327,0.04254818
0.2164887,0.02986137,0.01702112,0.01257615,0.010399057,0.009165145,0.008494327,0.008554401,0.04304601
0.4321415,0.08763481,0.05885976,0.04919712,0.044993039,0.043122980,0.042548176,0.043046013,0.00000000


In [16]:
# Comparing results to verify implementation/numerical accuracy
n1 <- 4
n2 <- 4

g <- construct_im_graph(n1,n2,0.1,0.1)

expected_visits <- rep(0, vertices_length(g))
ctx <- distribution_context(g,1000)
while (distribution_context_state(ctx)$cdf < 0.9999) {
  distribution_context_step(ctx)
}

expected_visits <- distribution_context_accumulated_visiting_time(ctx)

distribution_expectation <- matrix(nrow=n1+1,ncol=n2+1)
algorithm_expectation <- matrix(nrow=n1+1,ncol=n2+1)
matrix_expectation <- matrix(nrow=n1+1,ncol=n2+1)
simulation_expectation <- matrix(nrow=n1+1,ncol=n2+1)
PH <- graph_as_matrix(g)
U <- solve(-PH$SIM)
set.seed(1234)

for (i in 0:n1) {
  for (j in 0:n2) {
    distribution_expectation[i+1,j+1] <- sum(expected_visits * rewards_at(g, i,j,n1,n2))
    algorithm_expectation[i+1, j+1]<- expectation(g, rewards_at(g, i,j,n1,n2))
    simulation_expectation[i+1, j+1]<- mean(rph(10000, g, rewards_at(g, i,j,n1,n2)))
    matrix_expectation[i+1,j+1] <-PH$IPV %*% U%*%diag(PH$states[,(matrix_index(i,j,0,n1,n2)+1)]+PH$states[,(matrix_index(i,j,1,n1,n2)+1)])%*%rep(1,length(PH$IPV))
  }
}

sum(abs(matrix_expectation - algorithm_expectation))
sum(abs(matrix_expectation - simulation_expectation))
sum(abs(matrix_expectation - distribution_expectation))

[1] 1.214306e-14

[1] 0.3662765

[1] 0.001238273